#import libraties

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !pip install py7zr
!pip install datasets

In [ ]:
from google.colab import drive
import numpy as np
import os
import zipfile
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from transformers import PreTrainedTokenizer
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_linear_schedule_with_warmup
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.feature_extraction.text import CountVectorizer
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.models import resnet50
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from scipy.sparse import vstack
from transformers import Pix2StructVisionModel, AutoModelForCausalLM, AutoTokenizer
from transformers import AutoProcessor, Pix2StructVisionModel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
import numpy as np
# from sklearn.metrics import ndcg_score, reciprocal_rank_score

In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
print(torch.cuda.device_count())

1


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import dataset

In [ ]:
def load_data(directory_path):
  return load_dataset('arrow', data_files=directory_path)

In [ ]:
#Question_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Questions_Dialog/data-00000-of-00001.arrow') #contains dialog answer(only 394)
# Answer_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Answers Dialog/data-00000-of-00001.arrow') #just answerdialogs with question and answer
# Concat_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Concat_data/data-00000-of-00001.arrow') #contain both dialoganswer and questiondialog(one of them is null)
Question_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Questions_Dialog/data-00000-of-00001.arrow') #contains dialog answer(only 394)
# # Answer_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Answers Dialog/data-00000-of-00001.arrow') #just answerdialogs with question and answer
# # Concat_dialog_vqa = load_data('/content/drive/MyDrive/Datas/Concat_data/data-00000-of-00001.arrow') #contain both dialoganswer and questiondialog(one of them is null)

In [ ]:
columns_to_remove = ['ParentId', 'Score', 'ViewCount', 'ContentLicense',
                      'FavoriteCount', 'LastActivityDate', 'LastEditDate', 'LastEditorUserId', 'Tags','PostTypeId','AcceptedAnswerId','Title', 'CreationDate', 'OwnerUserId']
Question_dialog_vqa=Question_dialog_vqa['train'].remove_columns(columns_to_remove)
# Answer_dialog_vqa=Answer_dialog_vqa['train'].remove_columns(columns_to_remove)

In [ ]:
mydataset =Question_dialog_vqa.map(lambda example: {"Image": example["Image"][0]})

In [ ]:
print(mydataset['Image'][0:5])

['https://i.stack.imgur.com/h8Fod.png', 'https://i.stack.imgur.com/TvSPM.png', 'https://i.stack.imgur.com/irw1S.jpg', 'https://i.stack.imgur.com/lBFti.png', 'https://i.stack.imgur.com/GLgbU.jpg']


In [ ]:
print(mydataset)

Dataset({
    features: ['Id', 'Body', 'DialogHistory', 'Image', 'Answer', 'DialogAnswers'],
    num_rows: 8024
})


# define Model

In [ ]:
class FusionLayer(nn.Module):
    def __init__(self, image_encoder_output_size, text_encoder_output_size, hidden_size):
        super().__init__()
        self.attention = nn.Linear(image_encoder_output_size + text_encoder_output_size, 1)
        self.decoder = nn.LSTM(image_encoder_output_size + text_encoder_output_size, hidden_size, num_layers=2)
        self.linear =  nn.Linear(image_encoder_output_size + text_encoder_output_size, hidden_size)
        self.relu = nn.ReLU()


    # def forward(self, image_features, text_features):
    #     # Apply average pooling to the image features
    #     image_features = F.avg_pool1d(image_features.permute(0, 2, 1), kernel_size=2).permute(0, 2, 1)
    #     # Combine image and text features
    #     combined_features = torch.cat((image_features, text_features), dim=2)
    #     # Calculate attention weights
    #     attention_weights = torch.softmax(self.attention(combined_features), dim=1)

        # Apply attention to combined features
        # attended_features = torch.mul(combined_features, attention_weights)
        # fused_features =self.attention(attended_features)
        # fused_tensor_projected = self.relu(fused_features)
        # combined_features = fused_tensor_projected.reshape(-1, 1024)
        # # print(combined_features.shape)
        # # vocab_max = 50258  # Update with your actual maximum vocabulary value
        # # scaled_output = (combined_features ) * vocab_max / 2  # Scale between 0 and vocab_max
        # # Optional: You can round values to integers if needed
        # rounded_output = torch.round(combined_features).long()
        # # clipped_output = torch.clamp(rounded_output, 0, vocab_max - 1)
        # return  rounded_output

    def forward(self, image_features, text_features):
        # Apply average pooling to the image features
        image_features = F.avg_pool1d(image_features.permute(0, 2, 1), kernel_size=2).permute(0, 2, 1)
        # Combine image and text features
        combined_features = torch.cat((image_features, text_features), dim=2)
        # Calculate attention weights
        attention_weights = torch.softmax(self.attention(combined_features), dim=1)

        # Apply attention to combined features
        attended_features = torch.mul(combined_features, attention_weights)
        # fused_features =self.attention(attended_features)
        fused_tensor_projected = self.linear(attended_features)
        combined_features = fused_tensor_projected.reshape(-1, 1024)
        # print(combined_features.shape)
        fused_tensor_projected = torch.round(fused_tensor_projected ).long()
        return  fused_tensor_projected





class GenerativeVisualDialogModel(nn.Module):
    def __init__(self, image_encoder, dialog_encoder, question_encoder, decoder, fusion_layer):
        super(GenerativeVisualDialogModel, self).__init__()
        self.image_encoder = image_encoder
        self.dialog_encoder = dialog_encoder
        self.question_encoder = question_encoder
        self.decoder = decoder
        self.fusion_layer = fusion_layer

        num_layers_to_freeze = int(0.8 * 768)  # Assuming 768 is the total number of layers
        # Freeze the first layers of the image encoder
        num_layers_frozen_image = 0
        for child in self.image_encoder.children():
            for param in child.parameters():
                if num_layers_frozen_image < num_layers_to_freeze:
                    param.requires_grad = False
                    num_layers_frozen_image += 1
                else:
                    break
        # Freeze the first layers of the dialog encoder
        num_layers_to_freeze = int(0.8 * 1024)
        num_layers_frozen_dialog = 0
        for child in self.dialog_encoder.children():
            for param in child.parameters():
                if num_layers_frozen_dialog < num_layers_to_freeze:
                    param.requires_grad = False
                    num_layers_frozen_dialog += 1
                else:
                    break
        num_layers_to_freeze = int(0.8 * 1024)
        num_layers_frozen_dialog = 0
        for child in self.question_encoder.children():
                    for param in child.parameters():
                        if num_layers_frozen_dialog < num_layers_to_freeze:
                            param.requires_grad = False
                            num_layers_frozen_dialog += 1
                        else:
                            break


        num_layers_to_freeze = int(0.8 * 1024)
        num_layers_frozen_decoder = 0
        for child in self.decoder.children():
            for param in child.parameters():
                if num_layers_frozen_decoder < num_layers_to_freeze:
                    param.requires_grad = False
                    num_layers_frozen_dialog += 1
                else:
                    break

    def forward(self, images, questiondialogs,answersdialogs, questions, answers):
        # Encode images, dialogs, and questions
        val_probs = 0
        output = None
        outputs = self.image_encoder(**images)
        image_features =outputs.last_hidden_state
        dialog_features_questions = self.dialog_encoder(**questiondialogs)
        # dialog_features_answers = self.dialog_encoder(**answersdialogs)
        question_features = self.question_encoder(**questions)
        # print(dialog_features_questions.hidden_states[-2].shape,dialog_features_questions.hidden_states[-1].shape, dialog_features_questions.hidden_states[0].shape)
        text_features = torch.cat((dialog_features_questions.hidden_states[-2], question_features.hidden_states[-2]), dim=2)
        fused_features = self.fusion_layer( image_features, text_features)
        # Decode the fused features using the DialogGPT decoder

        # output = self.decoder.generate(input_ids = fused_features.to(torch.int32),max_length=50,pad_token_id=tokenizer.eos_token_id )
        # output = dialog_features_questions
       # inputs = torch.cat((torch.ones((fused_features.size(0), 1), dtype=torch.long).to(device) * tokenizer.bos_token_id, fused_features), dim=1)
        if(answers != None):

          labels = answers.reshape(-1, answers.shape[2])
          output = self.decoder(inputs_embeds =fused_features, labels=labels)
          # scores = output.logits
        if(answers is None):
            output = self.decoder.generate(inputs_embeds =fused_features, max_length=1024, pad_token_id=tokenizer.eos_token_id)

        # val_probs = torch.softmax(output.logits, dim=-1)[:, :, 1]

        return output,val_probs




In [ ]:
image_encoder = Pix2StructVisionModel.from_pretrained("google/pix2struct-textcaps-base")
text_LLM_encoder = AutoModelForCausalLM.from_pretrained( "/content/drive/MyDrive/Model/")
text_LLM_decoder = AutoModelForCausalLM.from_pretrained( "/content/drive/MyDrive/Model/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Pix2StructVisionModel were not initialized from the model checkpoint at google/pix2struct-textcaps-base and are newly initialized: ['embeddings.column_embedder.weight', 'embeddings.patch_projection.bias', 'embeddings.patch_projection.weight', 'embeddings.row_embedder.weight', 'encoder.layer.0.attention.key.weight', 'encoder.layer.0.attention.output.weight', 'encoder.layer.0.attention.query.weight', 'encoder.layer.0.attention.value.weight', 'encoder.layer.0.mlp.

In [ ]:
text_LLM_encoder.config.output_hidden_states =True
# text_LLM_decoder.config.output_hidden_states =True
print(text_LLM_encoder.config.output_hidden_states)

True


In [ ]:
text_LLM_encoder.config

GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/Model/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "output_hidden_states": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 50258
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a padding token
#  if tokenizer.pad_token is None:
# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a padding token
image_processor = AutoProcessor.from_pretrained("google/pix2struct-textcaps-base")
text_LLM_encoder.resize_token_embeddings(len(tokenizer))
text_LLM_decoder.resize_token_embeddings(len(tokenizer))

Embedding(50258, 1024)

In [ ]:

nltk.download('punkt')
nltk.download('stopwords')

def download_and_save_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            return image
        else:
            return None
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None


def simple_preprocess(text):

    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [word.lower() for word in tokens]
    processed_text = ' '.join(tokens)
    return processed_text


def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

def concat_nested_list(nested_list):
    if(nested_list == None):
      return ""
    flattened = flatten_list(nested_list)
    return ''.join(map(str, flattened))

def find_longest_sublist(main_list):
    longest_sublist = []
    for sublist in main_list:
        if len(sublist) > len(longest_sublist):
            longest_sublist = sublist
    return longest_sublist



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from transformers import AutoTokenizer
import torch
from PIL import Image
from transformers import AutoProcessor, Pix2StructVisionModel

# Define function to preprocess data
def preprocess_data(Q_dialog, A_dialog, Question ,image_path, answer):
    # Tokenize dialog
    Qdialog_tokens = tokenizer(Q_dialog, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    Adialog_tokens = tokenizer( A_dialog, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    # Load and process image
    image = Image.open(requests.get(image_path, stream=True).raw)
    image = image.convert("RGB")
    Question_tokens =  tokenizer(Question, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    answer_tokenized = tokenizer.encode(answer, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    return Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer_tokenized

# # Example usage
# dialog_tokens, image, answer_tokenized = preprocess_data("Sample dialog", "image.jpg", "Sample answer")


In [ ]:
# s1 = "this is just for test"


# s = tokenizer(s1, padding="max_length", truncation=True, max_length=1024, return_tensors="pt").to(device)
# M = text_LLM_encoder(**s)
# print(M.hidden_states[0].shape,M.hidden_states[-1].shape ,M.hidden_states[-2].shape ,M.hidden_states[1].shape , len(M.hidden_states)  )

In [ ]:
train_val_test_split = mydataset.train_test_split(test_size=0.995, shuffle=True)

# Split the trainval set into train and validation sets
train_val_split = train_val_test_split["train"].train_test_split(test_size=0.20, shuffle=True)

# Assign train, validation, and test sets
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = train_val_test_split["test"]

In [ ]:
print(train_dataset)

Dataset({
    features: ['Id', 'Body', 'DialogHistory', 'Image', 'Answer', 'DialogAnswers'],
    num_rows: 32
})


In [ ]:
print(val_dataset)

Dataset({
    features: ['Id', 'Body', 'DialogHistory', 'Image', 'Answer', 'DialogAnswers'],
    num_rows: 8
})


In [ ]:
print(test_dataset)

Dataset({
    features: ['Id', 'Body', 'DialogHistory', 'Image', 'Answer', 'DialogAnswers'],
    num_rows: 7984
})


In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.transform = transforms.Compose([
            transforms.Resize((768, 768)),  # Resize images to a fixed size
            transforms.ToTensor(),           # Convert images to PyTorch tensors
        ])


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # print("am I the problem??????????? ")

        ID, Question, Q_dialog, image_path, answer, A_dialog = self.data[idx]['Id'], self.data[idx]['Body'], self.data[idx]['DialogHistory'], self.data[idx]['Image'], self.data[idx]['Answer'], self.data[idx]['DialogAnswers']
        # print("am I the problem???????????????? ")
        print(ID, "Question:##",Question,"Dialogs:####", Q_dialog,"Image path:" ,image_path, "answer:#####",answer)
        if A_dialog is not None:
            # print("gv", A_dialog[0])
            A_dialog =   find_longest_sublist(A_dialog)
        if Q_dialog is not None:
            # print("g", Q_dialog[0])
            Q_dialog=  find_longest_sublist(Q_dialog)

        Q_dialog_string = concat_nested_list(Q_dialog)
        A_dialog_string = concat_nested_list(A_dialog)
        Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer_tokenized = preprocess_data(Q_dialog_string, A_dialog_string, Question, image_path, answer)
        image = self.transform(image)
        # Move tensors to device
        Qdialog_tokens = Qdialog_tokens.to(device)
        Adialog_tokens = Adialog_tokens.to(device)
        Question_tokens = Question_tokens.to(device)
        answer_tokenized = answer_tokenized.to(device)
        # image = image.to(device)

        # print(image)
        # print(image.size())
        return Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer_tokenized


trainDataset = MyDataset(train_dataset)
valDataset = MyDataset(val_dataset)
# Define data loader
batch_size = 4
data_loader = DataLoader(trainDataset , batch_size=batch_size, shuffle=True)
val_loader =  DataLoader(valDataset, batch_size=batch_size, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define model, optimizer, loss function
fusion_layer = FusionLayer(image_encoder_output_size=768, text_encoder_output_size=2048, hidden_size=text_LLM_decoder.config.n_embd)

model = GenerativeVisualDialogModel(image_encoder, text_LLM_encoder, text_LLM_encoder, text_LLM_decoder, fusion_layer)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

model.to(device)



GenerativeVisualDialogModel(
  (image_encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.06, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0-11): 12 x Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)
 

In [ ]:
text_LLM_decoder.config.n_embd

1024

In [ ]:
path = F"/content/drive/MyDrive/Final_Model/FILE_NAME"

In [ ]:
# checkpoint = torch.load(path, map_location=torch.device('cpu'))
checkpoint = torch.load(path)
# Print the keys to identify the correct key for the model state dictionary
print(checkpoint.keys())

# Load the model state_dict using the correct key
model.load_state_dict(checkpoint)

odict_keys(['image_encoder.embeddings.patch_projection.weight', 'image_encoder.embeddings.patch_projection.bias', 'image_encoder.embeddings.row_embedder.weight', 'image_encoder.embeddings.column_embedder.weight', 'image_encoder.encoder.layer.0.attention.query.weight', 'image_encoder.encoder.layer.0.attention.key.weight', 'image_encoder.encoder.layer.0.attention.value.weight', 'image_encoder.encoder.layer.0.attention.output.weight', 'image_encoder.encoder.layer.0.mlp.wi_0.weight', 'image_encoder.encoder.layer.0.mlp.wi_1.weight', 'image_encoder.encoder.layer.0.mlp.wo.weight', 'image_encoder.encoder.layer.0.pre_mlp_layer_norm.weight', 'image_encoder.encoder.layer.0.pre_attention_layer_norm.weight', 'image_encoder.encoder.layer.1.attention.query.weight', 'image_encoder.encoder.layer.1.attention.key.weight', 'image_encoder.encoder.layer.1.attention.value.weight', 'image_encoder.encoder.layer.1.attention.output.weight', 'image_encoder.encoder.layer.1.mlp.wi_0.weight', 'image_encoder.encoder.

<All keys matched successfully>

In [ ]:
data_loader = DataLoader(valDataset, batch_size=1, shuffle=True)

# Extract the first batch
first_batch = next(iter(data_loader))

Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer = first_batch

10650059 Question:## We have a file repository containing code files `*.sql; *.xep; *.dll; *.aspx; *.gif` that you are going to submit to Production pretty soon. In this repository we have the main folders which contain the most recent code files to go to PROD, but we also have all the Change Folders that where submitted to PRE-PROD with code. You can see the main repository structure in the image below:

![enter image description here](https://i.stack.imgur.com/Iode8.jpg)

The protocol is that whenever we submit something to PRE-PROD, we e create a Change Folder, place it in the main repository, and also update the main folders, but sometimes we forget to do the second part.
What i was trying to do in a automated way is: if there is a file with the same name in the main folder and the change folder they need the have the same modified date day (at least), this specific crossing i could easily do in Excel or even SQL.
So, finally :P , what i needed help in, is getting into a csv filefo

In [ ]:
# image = image_processor(images=image, return_tensors="pt").to(device)
i = 0
outputs = []
for Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer in data_loader:

  print(f"Datas {i}##################################################################")
  # image = image_processor(images=image, return_tensors="pt").to(device)
  with torch.no_grad():
      image = image_processor(images=image, return_tensors="pt").to(device)
      output,_ = model(image, Question_tokens, Adialog_tokens, Question_tokens, answer)
      print("######################################################################")
      i+=1
      outputs.append(output)




14022257 Question:## Eclipse is closing while loading any XML layout with this message `Java was started but returned exit code -1073741571`



```
-startup
plugins/org.eclipse.equinox.launcher_1.3.0.v20120522-1813.jar
--launcher.library
plugins/org.eclipse.equinox.launcher.win32.win32.x86_64_1.1.200.v20120522-1813
-product
com.android.ide.eclipse.adt.package.product
--launcher.XXMaxPermSize
256M
-showsplash
com.android.ide.eclipse.adt.package.product
--launcher.XXMaxPermSize
256m
--launcher.defaultAction
openFile
-vmargs
-Dosgi.requiredJavaVersion=1.6
-Xms512m
-Xmx1024m
-Xss1024k
-Declipse.buildId=v21.0.0-519525
```


![enter image description here](https://i.stack.imgur.com/KjkZ4.jpg)
 Dialogs:#### [['1360074 : There are at least 3 question with "Java was started but returned exit code" at SO. None of them help?', "1029916 : @NikolayKuznetsov Unless the exit code was the same I don't see them being very helpful to the OP. This http://support.microsoft.com/kb/838501 might be useful.",

In [ ]:
print(i)

8


In [ ]:
i = 0
for output in outputs:
    logits = output.logits

    # Convert logits to probabilities using softmax
    probabilities = torch.softmax(logits, dim=-1)

    # Get the index of the token with the highest probability (greedy decoding)
    generated_token_index = torch.argmax(probabilities, dim=-1)

    # Decode the generated token index using the tokenizer
    generated_answer = tokenizer.decode(generated_token_index[0], skip_special_tokens=True)
    print(f"answer {i}###########################################################")
    print("Generated answer:", generated_answer)
    print("######################################################################")
    i+=1

answer 0###########################################################
Generated answer: 10 </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject subject  subject           </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ </ 

In [ ]:
results = {
    'train_loss': [],
    'val_loss': [],
    'ndcg': [],
    'mrr': [],
    'model_state_dict': []  # to store model parameters
}
num_epochs_to_keep = 2

In [ ]:
# import warnings
# warnings.filterwarnings("ignore", message="Input length of input_ids is .* but `max_length` is .*")
# from torch.cuda.amp import GradScaler, autocast
# import numpy as np
# from sklearn.metrics import ndcg_score

# scaler = GradScaler()

# # Set model to train mode
# model.train()
# num_epochs = 5
# sequence_length = 1024
# best_val_loss = float('inf')
# patience = 4
# counter = 0
# values_loss = []
# values_loss_train = []
# values_NDCG = []

# for epoch in range(num_epochs):
#     running_train_loss = 0.0
#     running_val_loss = 0.0

#     # Training loop
#     for Qdialog_tokens, Adialog_tokens, Question_tokens, image, answer in data_loader:
#         optimizer.zero_grad()
#         image = image_processor(images=image, return_tensors="pt").to(device)
#         outputs ,_ = model(image, Question_tokens, Adialog_tokens, Question_tokens, answer)
#         # print(outputs.hidden_states[-2].shape)
#         loss = outputs.loss
#         loss.requires_grad = True
#         loss.backward()
#         nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         optimizer.step()
#         running_train_loss += loss.item()

#     # Compute average training loss
#     train_loss = running_train_loss / len(data_loader)
#     print(train_loss)

#     # Save model state
#     torch.save(model.state_dict(), path)

#     # Validation loop
#     # model.eval()
#     # all_ndcg_scores = []
#     # with torch.no_grad():
#     #     for val_Qdialog_tokens, val_Adialog_tokens, val_Question_tokens, val_image, val_answer in val_loader:
#     #         val_image = image_processor(images=val_image, return_tensors="pt").to(device)

#     #         val_outputs,predicted_scores  = model(val_image, val_Question_tokens, val_Adialog_tokens, val_Question_tokens, val_answer)

#     #         val_loss = val_outputs.loss
#     #         running_val_loss += val_loss.item()

#             # Compute NDCG batch-wise
#             # print(predicted_scores.shape)
#             # flattened_relevance_scores = val_answer.cpu().numpy().flatten()
#             # flattened_predicted_scores = predicted_scores.cpu().numpy().flatten()
#             # reshaped_relevance_scores = np.reshape(flattened_relevance_scores, (batch_size, sequence_length))
#             # reshaped_predicted_scores = np.reshape(flattened_predicted_scores, (batch_size, sequence_length))
#             # ndcg = ndcg_score(reshaped_relevance_scores, reshaped_predicted_scores)
#             # all_ndcg_scores.append(ndcg)

#     # Compute average validation loss
#     # val_loss = running_val_loss / len(val_loader)

#     # Compute average NDCG score
#     # avg_ndcg = np.mean(all_ndcg_scores)

#     # Print and store results
#     # values_loss.append(val_loss)
#     # values_loss_train.append(train_loss)
#     # # values_NDCG.append(avg_ndcg)
#     #print(f"Epoch {epoch+1}, Train Loss: {train_loss},  NDCG: avg_ndcg")

#     # # Check for early stopping
#     # if val_loss < best_val_loss:
#     #     best_val_loss = val_loss
#     #     counter = 0
#     # else:
#     #     counter += 1
#     #     if counter >= patience:
#     #         print("Validation loss did not improve for", patience, "epochs. Early stopping...")
#     #         break

#     # model.train()

# #print("Training finished.")


In [ ]:
# image_data , question_dialog_data , question_data  = []
# with torch.no_grad():
#     model.eval()  # Set the model to evaluation mode
#     output = model(images=image_data, questiondialogs=question_dialog_data, questions=question_data)

# # Process the output
# # For example, if your output is a tensor of predicted answers, you can convert it to text
# predicted_answers = output.argmax(dim=2)  # Assuming output is logits, take the argmax along the vocabulary dimension

# # Optionally, convert the predicted answers from token IDs to actual text using your tokenizer
# predicted_answer_texts = tokenizer.decode(predicted_answers)

# # Print or use the predicted answers as needed
# print("Predicted answers:", predicted_answers)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch


# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# # Let's chat for 5 lines
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     # generated a response while limiting the total chat history to 1000 tokens,
#     chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

#     # pretty print last ouput tokens from bot
#     print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


In [ ]:
# path = os.path.join(path,'/content/sample_data')
# torch.save(model,"/content/drive/MyDrive/model2/FILE_NAME')